In [27]:
import pandas as pd
import datetime

IRI_Selected_18=['LOS ANGELES','DALLAS/FT WORTH','NEW YORK','HOUSTON','WEST TEX/NEW MEXICO','PHOENIX/TUCSON','SOUTH CAROLINA',
                'MIAMI/FT LAUDERDALE','BALTIMORE/WASHINGTON','BIRMINGHAM/MONTGOMERY','NEW ORLEANS/MOBILE',
                 'CHICAGO','MISSISSIPPI','BOSTON','ATLANTA','DETROIT','RALEIGH/GREENSBORO','DENVER']

In [16]:
IRI_Counties_both=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='Sheet1',dtype=str)
IRI_Counties_IRI=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='IRI',dtype=str,usecols=[0,1])
IRI_Counties_Nielsen=pd.read_excel("/home/jian/SunnyD/Otherinput/DMA_to_IRI_Pctg/match_counties.xlsx",sheetname='Nielsen',dtype=str,usecols=[0,1])
IRI_Counties_Nielsen.columns=[['County','cleaned_name']]
IRI_Counties_both=IRI_Counties_both[(IRI_Counties_both['IRI_Label']!='nan') & (IRI_Counties_both['Nielsen_Label']!='nan')]

IRI_Counties_both=IRI_Counties_both[['County','County']]
IRI_Counties_both.columns=['County','cleaned_name']

Nielsen_counties=IRI_Counties_both.append(IRI_Counties_Nielsen).drop_duplicates().reset_index()
IRI_counties=IRI_Counties_both.append(IRI_Counties_IRI).drop_duplicates().reset_index()

del Nielsen_counties['index']
del IRI_counties['index']


In [17]:
All_DMA_zips=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
All_DMA_zips=All_DMA_zips.iloc[:,[0,2,6,9]]
All_DMA_zips.columns=['zip_cd','DMA','State','County']
All_DMA_zips['County']=All_DMA_zips['County']+"|"+All_DMA_zips['State']
All_DMA_zips['County']=All_DMA_zips['County'].apply(lambda x: x.replace(" ","").lower())
All_DMA_zips=All_DMA_zips[~All_DMA_zips['State'].isin(['AK','HI'])]

In [18]:
All_IRI_counties=pd.read_excel("/home/jian/SunnyD/Otherinput/Total_US_counties_IRI_markets_and_regions_cr_12272016.xlsx",dtype=str,sheetname='U.S. Counties',skiprows=1)
All_IRI_counties=All_IRI_counties[['County Name','State','IRI Market']]
All_IRI_counties['County']=All_IRI_counties['County Name']+"|"+All_IRI_counties['State']
All_IRI_counties['County']=All_IRI_counties['County'].apply(lambda x: x.replace(" ","").lower())
All_IRI_counties=pd.merge(All_IRI_counties,IRI_counties,on="County",how="left")
All_IRI_counties=All_IRI_counties[['IRI Market','cleaned_name']]

In [19]:
All_DMA_zips=pd.merge(All_DMA_zips,Nielsen_counties,on="County",how="left")
All_DMA_zips=All_DMA_zips[['zip_cd','DMA','County','cleaned_name']].rename(columns={"County":"Nielsen_County"})
All_DMA_zips_merge=pd.merge(All_DMA_zips,All_IRI_counties,on="cleaned_name",how="left")


In [21]:
IRI_Zip_with_dup=All_DMA_zips_merge[['IRI Market','zip_cd']].drop_duplicates()

In [50]:
df_demo=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/Demo_Dataset_2018EASI.csv",
                   dtype=str,usecols=['ZIP_CODE','POP18','HH18','Estimate; SEX AND AGE - 10 to 14 years','Estimate; SEX AND AGE - 15 to 19 years',
                                               'Estimate; Female: - 25 to 29 years','Estimate; Female: - 30 to 34 years','Estimate; Female: - 35 to 39 years',
                                               'Estimate; Female: - 40 to 44 years','Estimate; Female: - 45 to 49 years','Estimate; Female: - 50 to 54 years'])
for col in df_demo.columns.tolist()[1:]:
    df_demo[col]=df_demo[col].astype(float)
df_demo=df_demo.rename(columns={"ZIP_CODE":"zip_cd"})
df_demo['zip_cd']=df_demo['zip_cd'].apply(lambda x: x.zfill(5))

In [57]:
IRI_Pop_with_dup=pd.merge(IRI_Zip_with_dup,df_demo,on="zip_cd",how="left")
IRI_Pop_with_dup['Tee_Pop']=IRI_Pop_with_dup[['Estimate; SEX AND AGE - 10 to 14 years','Estimate; SEX AND AGE - 15 to 19 years']].sum(axis=1)
IRI_Pop_with_dup['Female_25_54_Pop']=IRI_Pop_with_dup[['Estimate; Female: - 25 to 29 years','Estimate; Female: - 30 to 34 years','Estimate; Female: - 35 to 39 years',
                                               'Estimate; Female: - 40 to 44 years','Estimate; Female: - 45 to 49 years','Estimate; Female: - 50 to 54 years']].sum(axis=1)



In [58]:
IRI_Pop_with_dup=IRI_Pop_with_dup.groupby(['IRI Market'])['POP18','HH18','Tee_Pop','Female_25_54_Pop'].sum().reset_index()
IRI_Pop_with_dup['18_Ind']=IRI_Pop_with_dup['IRI Market'].apply(lambda x: x in IRI_Selected_18)



In [60]:
IRI_Pop_with_dup.to_csv("/home/jian/SunnyD/Otherinput/Population_by_IRI/SunnyD_population by IRI with duplication_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

